In [80]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch import optim
from torch import autograd
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.utils.data as Data
from torch.autograd.variable import Variable
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split

%matplotlib inline

In [81]:
# parameters

batch_size = 3
num_epochs = 10
num_classes = 2
input_size = 67
learning_rate = 0.001
weight_decay = 0.0001

In [82]:
input_data = np.load('Adult/data.npy').astype('float32')
input_labels = np.load('Adult/labels.npy').astype('float32')



In [83]:
input_data.shape
input_labels.shape

(45222,)

In [84]:
train_data, test_data, train_labels, test_labels = train_test_split(input_data,input_labels,
                                                    test_size=0.20)

In [85]:

#Create a training set
tensor_x = torch.from_numpy(train_data).float()
tensor_y = torch.from_numpy(train_labels).long()
train_set = Data.TensorDataset(tensor_x,tensor_y)

#Create a test set
tensor_x = torch.from_numpy(test_data).float()
tensor_y = torch.from_numpy(test_labels).long()
test_set = Data.TensorDataset(tensor_x,tensor_y)


In [86]:
train_loader = Data.DataLoader(dataset=train_set, batch_size=batch_size, shuffle= True)

test_loader = Data.DataLoader(dataset = test_set, batch_size=batch_size, shuffle=False)


In [87]:
class Net(nn.Module):
    def __init__(self, input_size, num_classes):
        super(Net, self).__init__()
        self.fc1 = nn.Sequential(
            nn.Linear(input_size, 55),
            nn.ReLU())
        self.fc2 = nn.Sequential(
            nn.Linear(55, 35),
            nn.ReLU())
        self.fc3 = nn.Sequential(
            nn.Linear(35,num_classes))
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

In [88]:
def test():
    model.eval()
    test_acc = 0.0
    for i, (data, labels) in enumerate(test_loader):

        # Predict classes using images from the test set
        data = Variable(data)
        labels = Variable(labels)
        outputs = model(data)
        _, prediction = torch.max(outputs.data, 1)
        
#         prediction = prediction.data
#         prediction =
        test_acc = (i*test_acc +
                    torch.mean(prediction.eq(labels.data).float()))/(i+1)

    # Compute the average acc and loss over all 10000 test images
#     test_acc = test_acc / 10000

    return test_acc

In [89]:

# Create a learning rate adjustment function that divides the learning rate by 10 every 30 epochs
def adjust_learning_rate(epoch):
    lr = learning_rate

    if epoch > 100:
        lr = lr / 1000000
    elif epoch > 50:
        lr = lr / 100000
    elif epoch > 20:
        lr = lr / 10000
    elif epoch > 10:
        lr = lr / 1000
    elif epoch > 5:
        lr = lr / 100
    elif epoch > 2:
        lr = lr / 10

    for param_group in optimizer.param_groups:
        param_group["lr"] = lr

In [90]:
model = Net(input_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate, weight_decay= weight_decay)

In [91]:
def train(num_epochs):
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('-'*10)
        print("Epoch: {}/{} \n".format(epoch+1,num_epochs))
    #     losses = []
        train_acc = 0.0
        train_loss = 0.0
        model.train()
        for i ,(x, y) in enumerate(train_loader):
            optimizer.zero_grad()

            inputs =  Variable(x)
            output = model(inputs)
            targets = Variable(y.squeeze(-1))
            targets = targets.type(torch.LongTensor)

            loss = criterion(output, targets)
            loss.backward()
            optimizer.step()
            y = Variable(y)
            train_loss = (train_loss*i + loss.data[0])/(i+1)
            _, prediction = torch.max(output.data, 1)

            train_acc = (i*train_acc +
                 torch.mean(prediction.eq(y.data).float()))/(i+1)
    #         break
            adjust_learning_rate(epoch)
#     break
        # Evaluate on the test set
        test_acc = test()

        # Save the model if the test acc is greater than our current best
        if test_acc > best_acc:
#             save_models(epoch)
            best_acc = test_acc
            

        # Print the metrics
        print("Epoch {}, Train Accuracy: {} , TrainLoss: {:.4f} , Test Accuracy: {}".format(epoch+1, train_acc, train_loss,test_acc))
    print('\nbest_acc = ', best_acc)

In [92]:
if __name__ == "__main__":
    print("Started Training Network")
    train(num_epochs)
    print("\n\nFinished !!")

Started Training Network
----------
Epoch: 1/10 

Epoch 1, Train Accuracy: 0.8203278326008202 , TrainLoss: 0.3776 , Test Accuracy: 0.8309563294637912
----------
Epoch: 2/10 

Epoch 2, Train Accuracy: 0.8354479365342585 , TrainLoss: 0.3499 , Test Accuracy: 0.8357103372028735
----------
Epoch: 3/10 

Epoch 3, Train Accuracy: 0.8382674074688304 , TrainLoss: 0.3462 , Test Accuracy: 0.8323935876174674
----------
Epoch: 4/10 

Epoch 4, Train Accuracy: 0.8459518478591275 , TrainLoss: 0.3291 , Test Accuracy: 0.8425649530127115
----------
Epoch: 5/10 

Epoch 5, Train Accuracy: 0.8464217596815601 , TrainLoss: 0.3275 , Test Accuracy: 0.8452183526810376
----------
Epoch: 6/10 

Epoch 6, Train Accuracy: 0.8469193133758935 , TrainLoss: 0.3265 , Test Accuracy: 0.8449972360420102
----------
Epoch: 7/10 

Epoch 7, Train Accuracy: 0.8485225419465335 , TrainLoss: 0.3244 , Test Accuracy: 0.8447761194029831
----------
Epoch: 8/10 

Epoch 8, Train Accuracy: 0.8484119744589046 , TrainLoss: 0.3241 , Test Accu